# ground-truth-data

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

### Add id

In [5]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id


In [6]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [7]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

### Vefify uniqueness if id's/hashes

In [8]:
from collections import defaultdict

In [9]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [12]:
len(hashes), len(documents)

(947, 948)

In [14]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [16]:
hashes["593f7569"]

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

Question is same/duplicate for both hashes, so we can safely ignore this. Ideally, we should fix original dataset by merging duplicate questions etc.

## Generating ground truth data

In [17]:
import json

In [21]:
# save file
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [22]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [23]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [27]:
from openai import OpenAI
client = OpenAI(api_key='your_key')

### Let's try for 1 document first

In [28]:
doc = documents[2]
prompt = prompt_template.format(**doc)

print(prompt)

You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]


In [32]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

json_response = response.choices[0].message.content

json_response

'["Can I join the course after it starts?", "Am I eligible to submit homeworks without registration?", "Are there deadlines for final projects?", "Should I be aware of final project deadlines even if I join late?", "Can I turn in everything at the last minute?"]'

In [33]:
json.loads(json_response)

['Can I join the course after it starts?',
 'Am I eligible to submit homeworks without registration?',
 'Are there deadlines for final projects?',
 'Should I be aware of final project deadlines even if I join late?',
 'Can I turn in everything at the last minute?']

### Generate using all documents

In [29]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [30]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
results = {}

- To minimize costs, we are going to run below code block onlu for 10 senconds and stop execution. Check results and then simply use previously generated full result in next steps  

In [ ]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

In [36]:
results

{'c02e79ef': '["When will the course begin?", \n"What is the exact start date and time for the course?",\n"How do I subscribe to the course calendar?",\n"Where can I register before the course starts?",\n"How do I join the course announcement channel on Telegram?"]',
 '1f6520ca': '[\n  "What prerequisites should I meet for this course?",\n  "Are there any specific requirements before starting this course?",\n  "Where can I find the prerequisite details for this course?",\n  "Do I need any prior knowledge for this course?",\n  "Which skills are necessary before enrolling in this course?"\n]',
 '7842b56a': '[\n  "Can I register for the course after it has already started?",\n  "Is it possible to join the course if I miss the start date?",\n  "Will I be able to submit homeworks if I join the course late?",\n  "Are there deadlines for final projects if I join the course after it begins?",\n  "Can I participate in the course without registering at the beginning?"\n]',
 '0bbf41ec': '[\n    "

In [37]:
import pickle

In [38]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [40]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

In [47]:
print(json_questions)

[
"How can I resolve the Docker error 'invalid mode: \\Program Files\\Git\\var\\lib\\postgresql\\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
"How do I correct the mounting path error in Docker for \\\Program Files\\Git\\var\\lib\\postgresql\\data'?"
]


In [48]:
json_questions = [
"How can I resolve the Docker error 'invalid mode: \Program Files\Git\var\lib\postgresql\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
"How do I correct the mounting path error in Docker for \Program Files\Git\var\lib\postgresql\data'?"
]

In [50]:
results[doc_id] = json.dumps(json_questions)

In [51]:
parsed_result = {}

for doc_id, json_questions in results.items():
    parsed_result[doc_id] = json.loads(json_questions)

In [53]:
parsed_result['1f6520ca']

['Where can I find the prerequisites for this course?',
 'How do I check the prerequisites for this course?',
 'Where are the course prerequisites listed?',
 'What are the requirements for joining this course?',
 'Where is the list of prerequisites for the course?']

In [54]:
doc_index = {d['id']: d for d in documents}

In [55]:
final_results = []

for doc_id, questions in parsed_result.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [56]:
final_results[:10]

[('When does the course begin?', 'data-engineering-zoomcamp', 'c02e79ef'),
 ('How can I get the course schedule?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('What is the link for course registration?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('How can I receive course announcements?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Where do I join the Slack channel?',
  'data-engineering-zoomcamp',
  'c02e79ef'),
 ('Where can I find the prerequisites for this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('How do I check the prerequisites for this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Where are the course prerequisites listed?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('What are the requirements for joining this course?',
  'data-engineering-zoomcamp',
  '1f6520ca'),
 ('Where is the list of prerequisites for the course?',
  'data-engineering-zoomcamp',
  '1f6520ca')]

In [57]:
import pandas as pd

In [59]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [60]:
df

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
...,...,...,...
5083,How should I destroy infrastructure created us...,mlops-zoomcamp,886d1617
5084,What is the first step to destroy AWS infrastr...,mlops-zoomcamp,886d1617
5085,Can I destroy infrastructure created with GitH...,mlops-zoomcamp,886d1617
5086,What command initializes Terraform with specif...,mlops-zoomcamp,886d1617


In [61]:
df.to_csv('ground-truth-data.csv', index=False)